In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Training='/content/drive/MyDrive/numbers_recognition_Vit/'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2,
        rotation_range=30,
        zoom_range = 0.1,
        shear_range = 0.2,
        width_shift_range=0.122,
        height_shift_range=0.122,
        brightness_range = (0.5, 1.5))

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [ ]:
input_shape=(32,32,3)

In [ ]:
train_data = train_datagen.flow_from_directory(
   Training,
    subset="training",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode="categorical")

Found 2194 images belonging to 9 classes.


In [ ]:
valid_data = valid_datagen.flow_from_directory(
   Training,
    subset="validation",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode="categorical")

Found 544 images belonging to 9 classes.


In [ ]:
## imports for model
from tensorflow import keras
from keras.layers import Dropout, Flatten,Activation ,Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.applications import ResNet50,DenseNet121,MobileNetV2


In [ ]:
MobileNetV2=MobileNetV2(include_top=False,input_shape=(32, 32, 3))

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
Early_Stopping = EarlyStopping(
    monitor='val_loss', # Same as the
    patience=5, # Wait for 5 epochs before stopping the training
    verbose=1,
    restore_best_weights='True', # Restore model weights from the epoch with the best value of the monitored quantity.
    min_delta=0.1
)
reduce_on_plateau =ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x=MobileNetV2(inputs)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(9, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1, 1, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                             

In [ ]:
History = model.fit(train_data,validation_data =valid_data,epochs = 200,batch_size =32,callbacks=[reduce_on_plateau])

Epoch 1/200
69/69 [==============================] - 40s 122ms/step - loss: 0.5641 - accuracy: 0.8756 - val_loss: 0.9410 - val_accuracy: 0.9191 - lr: 0.0010
Epoch 2/200
69/69 [==============================] - 8s 122ms/step - loss: 0.3509 - accuracy: 0.9385 - val_loss: 0.0403 - val_accuracy: 0.9890 - lr: 0.0010
Epoch 3/200
69/69 [==============================] - 8s 112ms/step - loss: 0.3440 - accuracy: 0.9376 - val_loss: 30.4474 - val_accuracy: 0.6636 - lr: 0.0010
Epoch 4/200
69/69 [==============================] - 7s 105ms/step - loss: 0.2662 - accuracy: 0.9462 - val_loss: 13.3907 - val_accuracy: 0.8015 - lr: 0.0010
Epoch 5/200
69/69 [==============================] - 8s 121ms/step - loss: 0.2734 - accuracy: 0.9471 - val_loss: 3.5878 - val_accuracy: 0.8199 - lr: 0.0010
Epoch 6/200
69/69 [==============================] - ETA: 0s - loss: 0.3893 - accuracy: 0.9211
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
69/69 [==============================] - 8s 1

In [ ]:
model.save('/content/drive/MyDrive/mobile_numbers.h5')